# $\alpha$=0.1

In [5]:
import torch
import numpy as np
import torchvision.transforms as transforms               # include image preprocess tools
from torchvision.datasets import CIFAR100        # for loading images from Pytorch CIFAR
from torch.utils.data import DataLoader
import timm
import detectors
from src.raps import raps_scores, raps_classification, eval_aps, split_data_set
from src.temperature_scaling import ModelWithTemperature

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load fine-tuned model
model = timm.create_model("resnet18_cifar100", pretrained=True)
model = model.to(device)

# reprocess the images from CIFAR
data_transform = transforms.Compose([
    transforms.ToTensor(),          # transfer to tensor
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))  # normalize
])
# load images from CIFAR10
dataset = CIFAR100(root="../../data", train=False, download=True, transform=data_transform)

# temperature scaling
temp_scal_loader = DataLoader(dataset, batch_size=32, shuffle=True)
model = ModelWithTemperature(model, temperature=4.85).to(device)
model.set_temperature(temp_scal_loader)

model.eval()

# The number of times the experiment is going to be repeated
num_runs = 10

# error rate
alpha = 0.1
lambda_reg = 0.5
k_reg = 2

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("RAPS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # splite dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = raps_scores(model, calib_loader, alpha, lambda_reg, k_reg, device)
    t_cal = np.quantile(calib_scores, 1 - alpha)  # calculate 1-alpha quantile
    print(f"t_cal = {t_cal}")

    # construct APS
    aps, aps_labels, true_labels = raps_classification(model, test_loader, t_cal, lambda_reg, k_reg, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Ratein runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

Files already downloaded and verified
Before temperature - NLL: 1.123, ECE: 0.044
Optimal temperature: 4.704
After temperature - NLL: 3.095, ECE: 0.669
RAPS Classification, Start!

Running experiment 1/10...
t_cal = 1.1375144481658939
Total set size: 17566
Total coverage sets: 4501
Total samples amount: 5000
Average Prediction Set Size After APS in runs 1: 3.5132
Average Coverage Ratein runs 1: 0.9002

Running experiment 2/10...
t_cal = 1.1249503493309028
Total set size: 17211
Total coverage sets: 4489
Total samples amount: 5000
Average Prediction Set Size After APS in runs 2: 3.4422
Average Coverage Ratein runs 2: 0.8978

Running experiment 3/10...
t_cal = 1.5562797427177437
Total set size: 20014
Total coverage sets: 4518
Total samples amount: 5000
Average Prediction Set Size After APS in runs 3: 4.0028
Average Coverage Ratein runs 3: 0.9036

Running experiment 4/10...
t_cal = 1.5783290743827831
Total set size: 20151
Total coverage sets: 4532
Total samples amount: 5000
Average Predict

# $\alpha$=0.2

In [6]:
# error rate
alpha = 0.2
lambda_reg = 0.3
k_reg = 2

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("RAPS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # splite dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = raps_scores(model, calib_loader, alpha, lambda_reg, k_reg, device)
    t_cal = np.quantile(calib_scores, 1 - alpha)  # calculate 1-alpha quantile
    print(f"t_cal = {t_cal}")

    # construct APS
    aps, aps_labels, true_labels = raps_classification(model, test_loader, t_cal, lambda_reg, k_reg, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Ratein runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

RAPS Classification, Start!

Running experiment 1/10...
t_cal = 0.19665862917900093
Total set size: 8988
Total coverage sets: 4014
Total samples amount: 5000
Average Prediction Set Size After APS in runs 1: 1.7976
Average Coverage Ratein runs 1: 0.8028

Running experiment 2/10...
t_cal = 0.18612118363380453
Total set size: 8826
Total coverage sets: 3976
Total samples amount: 5000
Average Prediction Set Size After APS in runs 2: 1.7652
Average Coverage Ratein runs 2: 0.7952

Running experiment 3/10...
t_cal = 0.20290284156799324
Total set size: 9100
Total coverage sets: 4046
Total samples amount: 5000
Average Prediction Set Size After APS in runs 3: 1.82
Average Coverage Ratein runs 3: 0.8092

Running experiment 4/10...
t_cal = 0.19531791508197785
Total set size: 9019
Total coverage sets: 4011
Total samples amount: 5000
Average Prediction Set Size After APS in runs 4: 1.8038
Average Coverage Ratein runs 4: 0.8022

Running experiment 5/10...
t_cal = 0.20087815523147595
Total set size: 90

# $\alpha$=0.05

In [4]:
# error rate
alpha = 0.05
lambda_reg = 0.3
k_reg = 3

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("RAPS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # splite dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = raps_scores(model, calib_loader, alpha, lambda_reg, k_reg, device)
    t_cal = np.quantile(calib_scores, 1 - alpha)  # calculate 1-alpha quantile
    print(f"t_cal = {t_cal}")

    # construct APS
    aps, aps_labels, true_labels = raps_classification(model, test_loader, t_cal, lambda_reg, k_reg, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Ratein runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

RAPS Classification, Start!

Running experiment 1/10...
t_cal = 1.969767785072328
Total set size: 41291
Total coverage sets: 4749
Total samples amount: 5000
Average Prediction Set Size After APS in runs 1: 8.2582
Average Coverage Ratein runs 1: 0.9498

Running experiment 2/10...
t_cal = 1.9638457119464885
Total set size: 41058
Total coverage sets: 4746
Total samples amount: 5000
Average Prediction Set Size After APS in runs 2: 8.2116
Average Coverage Ratein runs 2: 0.9492

Running experiment 3/10...
t_cal = 1.9632853567600252
Total set size: 41067
Total coverage sets: 4746
Total samples amount: 5000
Average Prediction Set Size After APS in runs 3: 8.2134
Average Coverage Ratein runs 3: 0.9492

Running experiment 4/10...
t_cal = 1.9812520861625706
Total set size: 41719
Total coverage sets: 4755
Total samples amount: 5000
Average Prediction Set Size After APS in runs 4: 8.3438
Average Coverage Ratein runs 4: 0.951

Running experiment 5/10...
t_cal = 2.2586176037788395
Total set size: 454

# Result
  
$\alpha$=0.1
- Final Average **Prediction Set Size： 3.62**
- Final Average **Coverage: 90.04%**  

$\alpha$=0.2
- Final Average **Prediction Set Size： 1.80**
- Final Average **Coverage: 80.19%**  

$\alpha$=0.05
- Final Average **Prediction Set Size：8.27**
- Final Average **Coverage: 94.85%**